In [90]:
import warnings
warnings.filterwarnings('ignore')

In [91]:
#import useful libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
from random import randint
import time
from datetime import datetime
from functions import *

In [92]:
data = pd.read_csv('bank_transactions.csv', delimiter=',')

In [4]:
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


# Finding Similar Customers

## 1.1 Set up the date

clean the dataset

In [93]:
#check for any NA values
any(data.isna())
data.dropna(inplace=True)

convert string to datetime

In [94]:
#convert into datetime format
data.CustomerDOB = pd.to_datetime(data.CustomerDOB)
data.TransactionDate = pd.to_datetime(data.TransactionDate)

data.drop(data[data.CustomerDOB.dt.year > 1998].index, axis = 0, inplace = True)

data.drop(data[data.CustomerDOB == 1800].index, axis = 0, inplace=True)

data = data.reset_index()

We are going to keep only the rows We are going to use later in the query. Thus, we are going to get rid of *TransactionID* and *CustomerID*

In [95]:
data = data[['CustomerDOB','CustGender','CustLocation','CustAccountBalance','TransactionDate','TransactionTime','TransactionAmount (INR)']]

In [96]:
#get customer's age
data['CustomerDOB'] = data['CustomerDOB'].apply(lambda x: int(datetime.strftime(x, "%Y-%m-%w").split('-')[0]))
today = int(datetime.strftime(datetime.today().date(), "%Y-%m-%w").split('-')[0])
data['Age'] = data['CustomerDOB'].apply(lambda x: today - x)

In [97]:
#get weekday and hour in which transaction occurred
week_day = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
data['TransactionDay'] = data['TransactionTime'].apply(time.ctime)
data['Transaction_Hour'] = data['TransactionDay'].apply(lambda x : x[10:-4])
data['Transaction_weekday']     = data['TransactionDate'].apply(lambda x : week_day[pd.Timestamp(x).dayofweek])

In [98]:
#convert weekday into int value (e.g. Monday = 1)
data['Transaction_weekday'] = data['Transaction_weekday'].apply(lambda x: day_of_week(x))

#get the time range in which transaction happened (here time range is 2 hours long, e.g. 00:00:00-02:00:00)
data['Time_range'] = data['Transaction_Hour'].apply(lambda x: time_ranges(x))

In [99]:
data.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Age,TransactionDay,Transaction_Hour,Transaction_weekday,Time_range
0,1994,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0,28,Fri Jan 2 16:46:47 1970,16:46:47,1,9
1,1996,F,MUMBAI,17874.44,2016-02-08,142712,459.0,26,Fri Jan 2 16:38:32 1970,16:38:32,1,9
2,1973,F,MUMBAI,866503.21,2016-02-08,142714,2060.0,49,Fri Jan 2 16:38:34 1970,16:38:34,1,9
3,1988,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5,34,Sat Jan 3 03:19:16 1970,03:19:16,1,2
4,1972,F,ITANAGAR,53609.20,2016-02-08,173940,676.0,50,Sat Jan 3 01:19:00 1970,01:19:00,1,1


In [100]:
#after the feature engineering let's drop some columns
data = data.drop(['CustomerDOB','TransactionDate', 'TransactionTime', 'TransactionDay', 'Transaction_Hour'], axis = 1)

In [101]:
data.head()

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,Transaction_weekday,Time_range
0,F,JAMSHEDPUR,17819.05,25.0,28,1,9
1,F,MUMBAI,17874.44,459.0,26,1,9
2,F,MUMBAI,866503.21,2060.0,49,1,9
3,F,NAVI MUMBAI,6714.43,1762.5,34,1,2
4,F,ITANAGAR,53609.20,676.0,50,1,1


In [102]:
#we save a copy of the dataframe that we will use later
data_cp = data.copy(deep=True)

Convert values in int32 dtype in order to apply the hash function

In [103]:
data.CustGender = data['CustGender'].apply(string_to_int)
data.CustLocation = data['CustLocation'].apply(string_to_int)
data.CustAccountBalance = data['CustAccountBalance'].apply(float_to_int)
data['TransactionAmount (INR)'] = data['TransactionAmount (INR)'].apply(float_to_int)


In [104]:
data.head()

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,Transaction_weekday,Time_range
0,70,755,17819,25,28,1,9
1,70,443,17874,459,26,1,9
2,70,443,866503,2060,49,1,9
3,70,777,6714,1762,34,1,2
4,70,583,53609,676,50,1,1


In [105]:
#obtain max value for x
data.max()

CustGender                       84
CustLocation                   2609
CustAccountBalance         55369688
TransactionAmount (INR)     1560034
Age                             222
Transaction_weekday               7
Time_range                       12
dtype: int64

In [106]:
#let's get the next prime number bigger than x

from sympy import nextprime

max_value_of_x = nextprime(55369688)

## 1.2 Fingerprint Hashing

The hash function we want to implement is in the following form: <br>
$\begin{equation} h(x)=(ax+b)\mod{c}\end{equation}$ <br>
where: <br>
- *x*: input value
- *a*, *b*: randomly choosen integers less than the maximum value of *x*
- *c*: the next prime number bigger than the maximum value of *x*

Now it's time to implement the MinHash

In [107]:
c = max_value_of_x
M = 2**32 -1                           
n_hash = 10

a,b = random_coef(n_hash, M)

list_of_tuples = list(
    map(
        lambda x, y: (x, y),
        a,
        b
    )
)
data['min-hash']  = [min_hash([*x[1]],c,list_of_tuples) for x in (data.iterrows())]

In [108]:
data.head()

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,Transaction_weekday,Time_range,min-hash
0,70,755,17819,25,28,1,9,"[11040451, 678229, 2235825, 2821352, 2108599, ..."
1,70,443,17874,459,26,1,9,"[11040451, 8608876, 453510, 2563973, 437145, 6..."
2,70,443,866503,2060,49,1,9,"[11040451, 8608876, 3913906, 9336397, 7855981,..."
3,70,777,6714,1762,34,1,2,"[11040451, 3313423, 4774426, 6358841, 7601169,..."
4,70,583,53609,676,50,1,1,"[11040451, 13731951, 2690495, 5950415, 5678579..."


## 1.3 Locality Sensitive Hashing (LSH)

Now let's apply the same idea on the query, and then apply the Jaccard similarity since MinHash is stricly related to it

In [131]:
query = pd.read_csv('query_users.csv')

In [132]:
query.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,27/7/78,M,DELHI,94695.61,2/9/16,140310,65.0
1,6/11/92,M,PANCHKULA,7584.09,2/9/16,120214,6025.0
2,14/8/91,M,PATNA,7180.60,10/8/16,221732,541.5
3,3/1/87,M,CHENNAI,56847.75,29/8/16,144138,1000.0
4,4/1/95,M,GURGAON,84950.13,25/9/16,233309,80.0


pre-processing of the query

In [133]:
#convert date into datetime format
query.CustomerDOB = pd.to_datetime(query.CustomerDOB)
query.TransactionDate = pd.to_datetime(query.TransactionDate)

In [134]:
#get customer's age
query['CustomerDOB'] = query['CustomerDOB'].apply(lambda x: int(datetime.strftime(x, "%Y-%m-%w").split('-')[0]))
today = int(datetime.strftime(datetime.today().date(), "%Y-%m-%w").split('-')[0])
query['Age'] = query['CustomerDOB'].apply(lambda x: today - x)

In [135]:
#get weekday and transaction hour
week_day = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
query['TransactionDay'] = query['TransactionTime'].apply(time.ctime)
query['Transaction_Hour'] = query['TransactionDay'].apply(lambda x : x[10:-4])
query['Transaction_weekday'] = query['TransactionDate'].apply(lambda x : week_day[pd.Timestamp(x).dayofweek])

In [136]:
query.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Age,TransactionDay,Transaction_Hour,Transaction_weekday
0,1978,M,DELHI,94695.61,2016-02-09,140310,65.0,44,Fri Jan 2 15:58:30 1970,15:58:30,Tue
1,1992,M,PANCHKULA,7584.09,2016-02-09,120214,6025.0,30,Fri Jan 2 10:23:34 1970,10:23:34,Tue
2,1991,M,PATNA,7180.60,2016-10-08,221732,541.5,31,Sat Jan 3 14:35:32 1970,14:35:32,Sat
3,1987,M,CHENNAI,56847.75,2016-08-29,144138,1000.0,35,Fri Jan 2 17:02:18 1970,17:02:18,Mon
4,1995,M,GURGAON,84950.13,2016-09-25,233309,80.0,27,Sat Jan 3 17:48:29 1970,17:48:29,Sun


In [137]:
query['Transaction_weekday'] = query['Transaction_weekday'].apply(lambda x: day_of_week(x))
query['Time_range'] = query['Transaction_Hour'].apply(lambda x: time_ranges(x))

In [138]:
query = query.drop(['CustomerDOB','TransactionDate', 'TransactionTime', 'TransactionDay', 'Transaction_Hour'], axis = 1)

In [139]:
query_cp = query.copy(deep=True)

In [140]:
query_cp.head()

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,Transaction_weekday,Time_range
0,M,DELHI,94695.61,65.0,44,2,8
1,M,PANCHKULA,7584.09,6025.0,30,2,6
2,M,PATNA,7180.60,541.5,31,6,8
3,M,CHENNAI,56847.75,1000.0,35,1,9
4,M,GURGAON,84950.13,80.0,27,7,9


In [141]:

#convert other values into int32
query.CustGender = query['CustGender'].apply(string_to_int)
query.CustLocation = query['CustLocation'].apply(string_to_int)
query.CustAccountBalance = query['CustAccountBalance'].apply(float_to_int)
query['TransactionAmount (INR)'] = query['TransactionAmount (INR)'].apply(float_to_int)

In [121]:
query.head()

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,Transaction_weekday,Time_range
0,77,358,94695,65,44,2,8
1,77,663,7584,6025,30,2,6
2,77,372,7180,541,31,6,8
3,77,502,56847,1000,35,1,9
4,77,531,84950,80,27,7,9


In [142]:
query['min-hash']  = [min_hash([*x[1]],c,list_of_tuples) for x in (query.iterrows())]

In [123]:
query.head()

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,Transaction_weekday,Time_range,min-hash
0,77,358,94695,65,44,2,8,"[461559, 11899915, 3414210, 2356766, 4417968, ..."
1,77,663,7584,6025,30,2,6,"[461559, 1662786, 136784, 917563, 883958, 6497..."
2,77,372,7180,541,31,6,8,"[461559, 3469000, 3224676, 8970021, 1040441, 6..."
3,77,502,56847,1000,35,1,9,"[461559, 665009, 709613, 4960246, 11952089, 60..."
4,77,531,84950,80,27,7,9,"[461559, 1176192, 6006779, 5877961, 9526894, 3..."


Now let's compute Jaccard's similarity. <br>
Jaccard's similarity is defined as:
$J(A,B)=\dfrac{|A \cap B|}{|A \cup B|}$

Let's now compute the Jaccard Similarity between the query and the original dataframe on the minhash

In [124]:
resultDF = pd.DataFrame(columns=['Query Index', 'Data Index-Jaccard Similarity'])

In [189]:
#Test with a threshold set to 0.6
result = {}

start = time.time()
threshold = 0.6
for idx_1, val_1 in enumerate(query['min-hash']):
    jaccard_list = []
    index_list = []
    for idx_2, val_2 in enumerate(data['min-hash']):
        if jaccard(val_1, val_2) > threshold:
            index_list.append(idx_2)
            jaccard_list.append(jaccard(val_1, val_2))
            result.update({idx_1: sorted(list(zip(index_list,jaccard_list)), key = lambda x: x[1], reverse=True)})
end = time.time() - start

In [191]:
print(end)

86.84473609924316


In [180]:
resultDF['Query Index'] = list(result.keys())
resultDF['Data Index-Jaccard Similarity'] = list(result.values())

In [179]:
resultDF

,Query Index,Data Index-Jaccard Similarity
0,0,"[(687184, 1.0), (76578, 0.75)]"
1,1,"[(670151, 1.0), (419461, 0.75), (946865, 0.75)]"
2,2,"[(290921, 1.0)]"
3,3,"[(588909, 1.0), (736637, 0.625)]"
4,4,"[(10697, 1.0), (429598, 0.625), (837934, 0.625)]"
5,5,"[(862651, 1.0), (869679, 0.75)]"
6,6,"[(569281, 1.0), (139122, 0.75), (139345, 0.75)..."
7,7,"[(73174, 1.0), (210548, 0.75), (329966, 0.75),..."
8,8,"[(551951, 1.0), (561493, 0.75)]"
9,10,"[(116689, 1.0), (480784, 1.0), (193334, 0.75),..."


In [194]:
#Test with a threshold set to 0.7
result = {}
threshold = 0.7

start = time.time()
for idx_1, val_1 in enumerate(query['min-hash']):
    jaccard_list = []
    index_list = []
    for idx_2, val_2 in enumerate(data['min-hash']):
        if jaccard(val_1, val_2) > threshold:
            index_list.append(idx_2)
            jaccard_list.append(jaccard(val_1, val_2))
            result.update({idx_1: sorted(list(zip(index_list,jaccard_list)), key = lambda x: x[1], reverse=True)})
end = time.time() - start

In [195]:
end

87.40046095848083

In [184]:
len(result)

35

In [125]:
#Test with a threshold set to 0.8
result = {}

threshold = 0.8

start = time.time()
for idx_1, val_1 in enumerate(query['min-hash']):
    jaccard_list = []
    index_list = []
    for idx_2, val_2 in enumerate(data['min-hash']):
        if jaccard(val_1, val_2) > threshold:
            index_list.append(idx_2)
            jaccard_list.append(jaccard(val_1, val_2))
            result.update({idx_1: list(zip(index_list,jaccard_list))})

end = time.time() - start

In [126]:
end

85.3985641002655

In [127]:
result

{0: [(687184, 1.0)],
 1: [(670151, 1.0)],
 2: [(290921, 1.0)],
 3: [(588909, 1.0)],
 4: [(10697, 1.0)],
 5: [(862651, 1.0)],
 6: [(569281, 1.0)],
 7: [(73174, 1.0)],
 8: [(551951, 1.0)],
 10: [(116689, 1.0), (480784, 1.0)],
 11: [(271982, 1.0)],
 12: [(524003, 1.0)],
 13: [(256712, 1.0)],
 14: [(856165, 1.0)],
 15: [(647405, 1.0)],
 16: [(141610, 1.0)],
 17: [(172417, 1.0)],
 18: [(793697, 1.0), (794774, 1.0)],
 19: [(54776, 1.0)],
 20: [(496318, 1.0)],
 22: [(274894, 1.0)],
 24: [(75876, 1.0)],
 25: [(153452, 1.0)],
 26: [(6288, 1.0)],
 27: [(783549, 1.0)],
 28: [(204008, 1.0)],
 29: [(642811, 1.0), (645368, 1.0), (646953, 1.0)],
 30: [(232484, 1.0)],
 32: [(400663, 1.0)],
 33: [(777177, 1.0)],
 34: [(861726, 1.0)]}

In [143]:
#double-check
query_cp.iloc[0,:]

CustGender                        M
CustLocation                  DELHI
CustAccountBalance         94695.61
TransactionAmount (INR)        65.0
Age                              44
Transaction_weekday               2
Time_range                        8
Name: 0, dtype: object

In [144]:
data_cp.iloc[687184,:]

CustGender                        M
CustLocation                  DELHI
CustAccountBalance         94695.61
TransactionAmount (INR)        65.0
Age                              44
Transaction_weekday               2
Time_range                        8
Name: 687184, dtype: object